In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib

import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
file_path = '../../../data/BPI_Challenge_2019.xes'
event_log = pm4py.read_xes(file_path)

parsing log, completed traces ::   0%|          | 0/251734 [00:00<?, ?it/s]

In [3]:
event_log = TransformEventLog.seconds_in_day(event_log)
event_log = TransformEventLog.day_of_week(event_log)

In [4]:
known_activities = list(event_log['concept:name'].unique())
known_resources = list(event_log['org:resource'].unique())
print(len(known_resources))

628


In [5]:
known_activities = ['Block.Purchase.Order.Item', 'Cancel.Goods.Receipt', 'Cancel.Invoice.Receipt', 'Cancel.Subsequent.Invoice', 'Change.Approval.for.Purchase.Order', 'Change.Currency', 'Change.Delivery.Indicator', 'Change.Final.Invoice.Indicator', 'Change.Price', 'Change.Quantity', 'Change.Rejection.Indicator', 'Change.Storage.Location', 'Change.payment.term', 'Clear.Invoice', 'Create.Purchase.Order.Item', 'Create.Purchase.Requisition.Item', 'Delete.Purchase.Order.Item', 'Reactivate.Purchase.Order.Item', 'Receive.Order.Confirmation', 'Record.Goods.Receipt', 'Record.Invoice.Receipt', 'Record.Service.Entry.Sheet', 'Record.Subsequent.Invoice', 'Release.Purchase.Order', 'Release.Purchase.Requisition', 'Remove.Payment.Block', 'SRM..Awaiting.Approval', 'SRM..Change.was.Transmitted', 'SRM..Complete', 'SRM..Created', 'SRM..Deleted', 'SRM..Document.Completed', 'SRM..Held', 'SRM..In.Transfer.to.Execution.Syst.', 'SRM..Incomplete', 'SRM..Ordered', 'SRM..Transaction.Completed', 'SRM..Transfer.Failed..E.Sys..', 'Set.Payment.Block', 'Update.Order.Confirmation', 'Vendor.creates.debit.memo', 'Vendor.creates.invoice']
known_resources = ['NONE', 'batch_00', 'batch_01', 'batch_02', 'batch_03', 'batch_04', 'batch_05', 'batch_06', 'batch_07', 'batch_08', 'batch_09', 'batch_10', 'batch_11', 'batch_12', 'batch_13', 'batch_14', 'batch_15', 'batch_16', 'batch_17', 'batch_18', 'batch_19', 'user_000', 'user_001', 'user_002', 'user_003', 'user_004', 'user_005', 'user_006', 'user_007', 'user_008', 'user_009', 'user_010', 'user_011', 'user_012', 'user_013', 'user_014', 'user_015', 'user_016', 'user_017', 'user_018', 'user_019', 'user_020', 'user_021', 'user_022', 'user_023', 'user_024', 'user_025', 'user_026', 'user_027', 'user_028', 'user_029', 'user_030', 'user_031', 'user_032', 'user_033', 'user_034', 'user_035', 'user_036', 'user_037', 'user_038', 'user_039', 'user_040', 'user_041', 'user_042', 'user_043', 'user_044', 'user_045', 'user_046', 'user_047', 'user_048', 'user_049', 'user_050', 'user_051', 'user_052', 'user_053', 'user_054', 'user_055', 'user_056', 'user_057', 'user_058', 'user_059', 'user_060', 'user_061', 'user_062', 'user_063', 'user_064', 'user_065', 'user_066', 'user_067', 'user_068', 'user_069', 'user_070', 'user_071', 'user_072', 'user_073', 'user_074', 'user_075', 'user_076', 'user_077', 'user_078', 'user_079', 'user_080', 'user_081', 'user_082', 'user_083', 'user_084', 'user_085', 'user_086', 'user_087', 'user_088', 'user_089', 'user_090', 'user_091', 'user_092', 'user_093', 'user_094', 'user_095', 'user_096', 'user_097', 'user_098', 'user_099', 'user_100', 'user_101', 'user_102', 'user_103', 'user_104', 'user_105', 'user_106', 'user_107', 'user_108', 'user_109', 'user_110', 'user_111', 'user_112', 'user_113', 'user_114', 'user_115', 'user_116', 'user_117', 'user_118', 'user_119', 'user_120', 'user_121', 'user_122', 'user_123', 'user_124', 'user_125', 'user_126', 'user_127', 'user_128', 'user_129', 'user_130', 'user_131', 'user_132', 'user_133', 'user_134', 'user_135', 'user_136', 'user_137', 'user_138', 'user_139', 'user_140', 'user_141', 'user_142', 'user_143', 'user_144', 'user_145', 'user_146', 'user_147', 'user_148', 'user_149', 'user_150', 'user_151', 'user_152', 'user_153', 'user_154', 'user_155', 'user_156', 'user_157', 'user_158', 'user_159', 'user_160', 'user_161', 'user_162', 'user_163', 'user_164', 'user_165', 'user_166', 'user_167', 'user_168', 'user_169', 'user_170', 'user_171', 'user_172', 'user_173', 'user_174', 'user_175', 'user_176', 'user_177', 'user_178', 'user_179', 'user_180', 'user_181', 'user_182', 'user_183', 'user_184', 'user_185', 'user_186', 'user_187', 'user_188', 'user_189', 'user_190', 'user_191', 'user_192', 'user_193', 'user_194', 'user_195', 'user_196', 'user_197', 'user_198', 'user_199', 'user_200', 'user_201', 'user_202', 'user_203', 'user_204', 'user_205', 'user_206', 'user_207', 'user_208', 'user_209', 'user_210', 'user_211', 'user_212', 'user_213', 'user_214', 'user_215', 'user_216', 'user_217', 'user_218', 'user_219', 'user_220', 'user_221', 'user_222', 'user_223', 'user_224', 'user_225', 'user_226', 'user_227', 'user_228', 'user_229', 'user_230', 'user_231', 'user_232', 'user_233', 'user_234', 'user_235', 'user_236', 'user_237', 'user_238', 'user_239', 'user_240', 'user_241', 'user_242', 'user_243', 'user_244', 'user_245', 'user_246', 'user_247', 'user_248', 'user_249', 'user_250', 'user_251', 'user_252', 'user_253', 'user_254', 'user_255', 'user_256', 'user_257', 'user_258', 'user_259', 'user_260', 'user_261', 'user_262', 'user_263', 'user_264', 'user_265', 'user_266', 'user_267', 'user_268', 'user_269', 'user_270', 'user_271', 'user_272', 'user_273', 'user_274', 'user_275', 'user_277', 'user_278', 'user_279', 'user_280', 'user_281', 'user_282', 'user_283', 'user_284', 'user_285', 'user_286', 'user_287', 'user_288', 'user_289', 'user_290', 'user_291', 'user_292', 'user_293', 'user_294', 'user_295', 'user_296', 'user_297', 'user_298', 'user_299', 'user_300', 'user_301', 'user_302', 'user_303', 'user_304', 'user_305', 'user_306', 'user_307', 'user_308', 'user_309', 'user_310', 'user_311', 'user_312', 'user_313', 'user_314', 'user_315', 'user_316', 'user_317', 'user_318', 'user_319', 'user_320', 'user_321', 'user_322', 'user_323', 'user_324', 'user_325', 'user_326', 'user_327', 'user_328', 'user_329', 'user_330', 'user_331', 'user_332', 'user_333', 'user_334', 'user_335', 'user_336', 'user_337', 'user_338', 'user_339', 'user_340', 'user_341', 'user_342', 'user_343', 'user_344', 'user_345', 'user_346', 'user_347', 'user_348', 'user_349', 'user_350', 'user_351', 'user_352', 'user_353', 'user_354', 'user_355', 'user_356', 'user_357', 'user_358', 'user_359', 'user_360', 'user_361', 'user_362', 'user_363', 'user_364', 'user_365', 'user_366', 'user_367', 'user_368', 'user_369', 'user_370', 'user_371', 'user_372', 'user_373', 'user_374', 'user_375', 'user_376', 'user_377', 'user_378', 'user_379', 'user_380', 'user_381', 'user_382', 'user_383', 'user_384', 'user_385', 'user_386', 'user_387', 'user_388', 'user_389', 'user_390', 'user_391', 'user_392', 'user_393', 'user_394', 'user_396', 'user_397', 'user_398', 'user_399', 'user_400', 'user_401', 'user_402', 'user_403', 'user_404', 'user_405', 'user_406', 'user_407', 'user_409', 'user_410', 'user_411', 'user_412', 'user_413', 'user_414', 'user_415', 'user_416', 'user_417', 'user_418', 'user_419', 'user_420', 'user_421', 'user_423', 'user_424', 'user_425', 'user_427', 'user_428', 'user_429', 'user_430', 'user_431', 'user_432', 'user_433', 'user_434', 'user_435', 'user_436', 'user_437', 'user_438', 'user_439', 'user_440', 'user_441', 'user_442', 'user_444', 'user_445', 'user_446', 'user_447', 'user_448', 'user_449', 'user_450', 'user_451', 'user_452', 'user_453', 'user_454', 'user_455', 'user_456', 'user_457', 'user_458', 'user_459', 'user_460', 'user_461', 'user_462', 'user_463', 'user_464', 'user_465', 'user_466', 'user_467', 'user_468', 'user_469', 'user_470', 'user_471', 'user_472', 'user_473', 'user_474', 'user_475', 'user_476', 'user_477', 'user_478', 'user_479', 'user_480', 'user_481', 'user_482', 'user_483', 'user_484', 'user_485', 'user_486', 'user_487', 'user_488', 'user_489', 'user_490', 'user_491', 'user_492', 'user_493', 'user_494', 'user_495', 'user_496', 'user_497', 'user_498', 'user_499', 'user_500', 'user_501', 'user_502', 'user_503', 'user_504', 'user_505', 'user_506', 'user_507', 'user_508', 'user_509', 'user_510', 'user_511', 'user_512', 'user_513', 'user_514', 'user_515', 'user_516', 'user_517', 'user_518', 'user_519', 'user_520', 'user_521', 'user_522', 'user_523', 'user_524', 'user_525', 'user_526', 'user_527', 'user_528', 'user_529', 'user_530', 'user_531', 'user_532', 'user_533', 'user_534', 'user_535', 'user_536', 'user_537', 'user_538', 'user_539', 'user_540', 'user_541', 'user_542', 'user_543', 'user_544', 'user_545', 'user_546', 'user_547', 'user_548', 'user_549', 'user_550', 'user_551', 'user_552', 'user_553', 'user_554', 'user_555', 'user_556', 'user_557', 'user_558', 'user_559', 'user_560', 'user_561', 'user_562', 'user_563', 'user_564', 'user_565', 'user_566', 'user_567', 'user_568', 'user_569', 'user_570', 'user_571', 'user_572', 'user_573', 'user_574', 'user_575', 'user_576', 'user_577', 'user_578', 'user_579', 'user_580', 'user_581', 'user_582', 'user_583', 'user_584', 'user_585', 'user_586', 'user_587', 'user_588', 'user_589', 'user_590', 'user_591', 'user_592', 'user_593', 'user_594', 'user_595', 'user_597', 'user_598', 'user_599', 'user_601', 'user_602', 'user_603', 'user_604', 'user_605', 'user_606']
print(len(known_resources))

620


In [6]:
drbart_model = DRBART(parser_dir = '../../../models/external/bpic_2019_2/concept-name_resource_seconds-in-day_day-of-week_activity-count_resoure-count/',
                     strict_parser=False)
sample_drbart = SampleOutcomes_DRBART_Normal_R_A_S_D_AC_RC(event_log, drbart_model, known_activities, known_resources)

In [7]:
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity

importlib.reload(sys.modules['normal_evaluation.drbart_evaluation'])
importlib.reload(sys.modules['normal_evaluation.normal_evaluation'])
from normal_evaluation.normal_evaluation import *
from normal_evaluation.drbart_evaluation import *

kde_likelihood = lambda samples, ground_truth : gaussian_kde(samples).pdf(ground_truth) * (3600*24)
kde_likelihood3 = lambda samples, ground_truth : np.exp(
                                                        KernelDensity(kernel='linear')\
                                                            .fit(np.expand_dims(np.array(samples), axis=1)).score([[ground_truth]])
                                                    ) * (3600*24)

get_real_end_time = lambda case_name : event_log[event_log['case:concept:name'] == case_name]['time:timestamp'].max().timestamp()

results = []
n = 1
for case_name in event_log['case:concept:name'].unique():
    case_log = event_log[event_log['case:concept:name'] == case_name]
    print(case_log)
    real_end_time = get_real_end_time(case_name)
    print(real_end_time)

    case_samples = [sample_drbart.sample_case(case_name) for i in range(n)]
    likelihood = kde_likelihood(case_samples, real_end_time)
    likelihood3 = kde_likelihood3(case_samples, real_end_time)

    
    case_samples = [datetime.datetime.fromtimestamp(ts) for ts in case_samples]
    real_end_time = datetime.datetime.fromtimestamp(real_end_time)

    plt.figure(figsize=(14, 6))
    #plt.xticks( rotation=25 )
    #print(case_samples)
    ax=plt.gca()
    xfmt = md.DateFormatter('%Y-%m-%d')
    ax.xaxis.set_major_formatter(xfmt)
    ax.xaxis.set_major_locator(md.WeekdayLocator(byweekday=md.SU, interval=1))
    mxfmt = md.DateFormatter('%a')
    ax.xaxis.set_minor_formatter(mxfmt)
    ax.xaxis.set_minor_locator(md.DayLocator())
    ax.tick_params(axis='both', which='major', labelsize=8, rotation=90)
    ax.tick_params(axis='both', which='minor', labelsize=5, rotation=90)
    
    plt.gcf().autofmt_xdate()
    plt.gcf().set_dpi(300)
    plt.hist(case_samples, bins=pandas.date_range(start= pandas.to_datetime(min(case_samples)).floor('D'), end=pandas.to_datetime(max(case_samples)).ceil('D'), freq='6H'), density=True)
    plt.axvline(x=real_end_time, color='red')
    plt.show()
    results.append(likelihood[0])
    print(case_name, likelihood, likelihood3, min(case_samples), max(case_samples),  max(case_samples)-min(case_samples))
    break


        User org:resource                         concept:name  \
0   batch_00     batch_00                         SRM: Created   
1   batch_00     batch_00                        SRM: Complete   
2   batch_00     batch_00               SRM: Awaiting Approval   
3   batch_00     batch_00              SRM: Document Completed   
4   batch_00     batch_00  SRM: In Transfer to Execution Syst.   
5   batch_00     batch_00                         SRM: Ordered   
6   batch_00     batch_00          SRM: Change was Transmitted   
7   user_000     user_000           Create Purchase Order Item   
8       NONE         NONE               Vendor creates invoice   
9   user_000     user_000                 Record Goods Receipt   
10  user_001     user_001               Record Invoice Receipt   
11  user_002     user_002                        Clear Invoice   

    Cumulative net worth (EUR)            time:timestamp case:Spend area text  \
0                        298.0 2018-01-02 12:53:00+00:00    

ValueError: year 25041788 is out of range

In [ ]:
sys.modules['normal_evaluation.drbart_evaluation']

In [ ]:
a = ['batch_00', 'SRM: Created', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
len(a)

In [ ]:
for i, j in zip(range(len(a)), a):
    print(i, j)
    print(drbart_model.parser.encoding_name_to_id[i][j])

In [ ]:
len(known_activities)

In [ ]:
len(known_resources)

In [ ]:
len(known_activities)+len(known_resources)

In [ ]:
len(drbart_model.parser.encoding_name_to_id)

In [ ]:
drbart_model.parser.encoding_name_to_id

In [ ]:
drbart_model.parser.encoding_id_to_name